In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=False)

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import csv
from imutils import paths
import os
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.applications import MobileNetV2
#from tensorflow.keras.applications import ResNet50V2
#from tensorflow.keras.applications import InceptionV3
#from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
#from tensorflow.keras.applications.resnet_v2 import preprocess_input
#from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.densenet import preprocess_input
#from tensorflow.keras.applications.vgg16 import preprocess_input
from time import time
from keras.callbacks import TensorBoard

In [ ]:
INIT_LR = 1e-4
num_epochs = 25
BS = 32

In [ ]:
def create_model():
  baseModel = DenseNet121(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

  headModel = baseModel.output
  headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
  headModel = Flatten(name="flatten")(headModel)
  headModel = Dense(128, activation="relu")(headModel)
  headModel = Dropout(0.5)(headModel)
  headModel = Dense(2, activation="softmax")(headModel)

  model = Model(inputs=baseModel.input, outputs=headModel)

  for layer in baseModel.layers:
      layer.trainable = False
      #print("running")

  return model

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/training_labels.csv')
Y = train_data[['label']]

kf = KFold(n_splits = 5)
                         
skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True)

In [ ]:
# idg = ImageDataGenerator(width_shift_range=0.1,
#                          height_shift_range=0.1,
#                          zoom_range=0.3,
#                          fill_mode='nearest',
#                          horizontal_flip = True,
#                          rescale=1./255)

idg = ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")

In [ ]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [ ]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

save_dir = '/content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/'
fold_var = 1
image_dir = "/content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/ALL_DATASETS/dataset"
n=3846 # total number of images

In [ ]:
for train_index, val_index in skf.split(np.zeros(n),Y):
  
  training_data = train_data.iloc[train_index]
  validation_data = train_data.iloc[val_index]
	
  train_data_generator = idg.flow_from_dataframe(training_data, directory = image_dir,x_col = "filename", y_col = "label", class_mode = "categorical", shuffle = True,batch_size=BS)
  valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = image_dir,x_col = "filename", y_col = "label",class_mode = "categorical", shuffle = True)
 
	
  model = create_model()
  model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=INIT_LR, decay=INIT_LR / num_epochs),metrics=['accuracy'])
 
  tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/logs/{}'.format(time()))
  
  checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var),monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
  tensorboard = TensorBoard(log_dir='/content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/logs/{}'.format(time()))
  callbacks_list = [checkpoint,tensorboard]

  print('\nFold ',fold_var)
  history = model.fit(train_data_generator,epochs=num_epochs,callbacks=[callbacks_list],validation_data=valid_data_generator)
 
  model.load_weights("/content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/model_"+str(fold_var)+".h5")
	
  results = model.evaluate(valid_data_generator)
  results = dict(zip(model.metrics_names,results))
	
  VALIDATION_ACCURACY.append(results['accuracy'])
  VALIDATION_LOSS.append(results['loss'])
	
  tf.keras.backend.clear_session()
	
  fold_var += 1

Found 3076 validated image filenames belonging to 2 classes.
Found 770 validated image filenames belonging to 2 classes.
29089792/29084464 [==============================] - 0s 0us/step

Fold  1
Epoch 1/25
57/97 [================>.............] - ETA: 5:38 - loss: 2.4037 - accuracy: 0.5579

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


97/97 [==============================] - 1076s 11s/step - loss: 1.9981 - accuracy: 0.5920 - val_loss: 0.7954 - val_accuracy: 0.7143

Epoch 00001: val_accuracy improved from -inf to 0.71429, saving model to /content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/model_1.h5
Epoch 2/25
97/97 [==============================] - 61s 626ms/step - loss: 0.9681 - accuracy: 0.6902 - val_loss: 0.5540 - val_accuracy: 0.7584

Epoch 00002: val_accuracy improved from 0.71429 to 0.75844, saving model to /content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/model_1.h5
Epoch 3/25
97/97 [==============================] - 61s 626ms/step - loss: 0.6525 - accuracy: 0.7461 - val_loss: 0.5164 - val_accuracy: 0.7818

Epoch 00003: val_accuracy improved from 0.75844 to 0.78182, saving model to /content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/model_1.h5
Epoch 4/25
97/97 [==============================] - 61s 625ms/step - loss: 0.5439 - accuracy: 0.7585 - val_loss: 0

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


97/97 [==============================] - 71s 678ms/step - loss: 1.5478 - accuracy: 0.6201 - val_loss: 0.6369 - val_accuracy: 0.7035

Epoch 00001: val_accuracy improved from -inf to 0.70351, saving model to /content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/model_2.h5
Epoch 2/25
97/97 [==============================] - 61s 629ms/step - loss: 0.7743 - accuracy: 0.7108 - val_loss: 0.5215 - val_accuracy: 0.7672

Epoch 00002: val_accuracy improved from 0.70351 to 0.76723, saving model to /content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/model_2.h5
Epoch 3/25
97/97 [==============================] - 61s 630ms/step - loss: 0.5968 - accuracy: 0.7475 - val_loss: 0.4786 - val_accuracy: 0.7750

Epoch 00003: val_accuracy improved from 0.76723 to 0.77503, saving model to /content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/model_2.h5
Epoch 4/25
97/97 [==============================] - 61s 633ms/step - loss: 0.5484 - accuracy: 0.7533 - val_loss: 0

...

97/97 [==============================] - 69s 655ms/step - loss: 1.6904 - accuracy: 0.5860 - val_loss: 0.6353 - val_accuracy: 0.6983

Epoch 00001: val_accuracy improved from -inf to 0.69831, saving model to /content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/model_3.h5
Epoch 2/25
97/97 [==============================] - 61s 628ms/step - loss: 0.8324 - accuracy: 0.6854 - val_loss: 0.5169 - val_accuracy: 0.7607

Epoch 00002: val_accuracy improved from 0.69831 to 0.76073, saving model to /content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/model_3.h5
Epoch 3/25
97/97 [==============================] - 61s 633ms/step - loss: 0.6401 - accuracy: 0.7221 - val_loss: 0.4585 - val_accuracy: 0.7906

Epoch 00003: val_accuracy improved from 0.76073 to 0.79064, saving model to /content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/model_3.h5
Epoch 4/25
97/97 [==============================] - 62s 639ms/step - loss: 0.5571 - accuracy: 0.7465 - val_loss: 0

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


97/97 [==============================] - 69s 652ms/step - loss: 1.6388 - accuracy: 0.6003 - val_loss: 0.6315 - val_accuracy: 0.7516

Epoch 00001: val_accuracy improved from -inf to 0.75163, saving model to /content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/model_4.h5
Epoch 2/25
97/97 [==============================] - 61s 625ms/step - loss: 0.8135 - accuracy: 0.6974 - val_loss: 0.5322 - val_accuracy: 0.7503

Epoch 00002: val_accuracy did not improve from 0.75163
Epoch 3/25
97/97 [==============================] - 60s 623ms/step - loss: 0.6287 - accuracy: 0.7179 - val_loss: 0.5121 - val_accuracy: 0.7659

Epoch 00003: val_accuracy improved from 0.75163 to 0.76593, saving model to /content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/model_4.h5
Epoch 4/25
97/97 [==============================] - 61s 624ms/step - loss: 0.5578 - accuracy: 0.7446 - val_loss: 0.4689 - val_accuracy: 0.7945

Epoch 00004: val_accuracy improved from 0.76593 to 0.79454, saving mode

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


97/97 [==============================] - 69s 651ms/step - loss: 1.7585 - accuracy: 0.5954 - val_loss: 0.5975 - val_accuracy: 0.7269

Epoch 00001: val_accuracy improved from -inf to 0.72692, saving model to /content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/model_5.h5
Epoch 2/25
97/97 [==============================] - 69s 714ms/step - loss: 0.8469 - accuracy: 0.6848 - val_loss: 0.4897 - val_accuracy: 0.7776

Epoch 00002: val_accuracy improved from 0.72692 to 0.77763, saving model to /content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/model_5.h5
Epoch 3/25
97/97 [==============================] - 61s 624ms/step - loss: 0.6249 - accuracy: 0.7199 - val_loss: 0.4674 - val_accuracy: 0.7906

Epoch 00003: val_accuracy improved from 0.77763 to 0.79064, saving model to /content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/model_save/model_5.h5
Epoch 4/25
97/97 [==============================] - 61s 627ms/step - loss: 0.5480 - accuracy: 0.7517 - val_loss: 0

##Extra Code Snippets

###CSV File generate code from dataset

In [ ]:
# imagePaths = list(paths.list_images('/content/drive/MyDrive/Colab Notebooks/498R/Mask Detector/dataset_labeled'))

# with open('training_labels.csv', 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(["filename","label"])
#     for imagePath in imagePaths:

#         image_name = imagePath.split(os.path.sep)[-1] 
#         label = imagePath.split(os.path.sep)[-2]

#         writer = csv.writer(file)
#         writer.writerow([image_name, label])